In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import statsmodels
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

# Data Loading and preparing for viz

In [ ]:
df = pd.read_csv('/kaggle/input/adhaar-biometric-registration-2025/cleaned_df.csv')
df.head()

In [ ]:
df.shape

## Null values

In [ ]:
df.isnull().sum()

## Trying to clean nan pincode values with mapping in the data - dropped  

In [ ]:
nan_pincodes = df[df.district_clean.isna()][['state_clean','district_clean','pincode']].reset_index(drop=True)

nan_pincodes.head()

In [ ]:
df[df.district_clean.isna()][['state_clean','district_clean','pincode']].groupby('state_clean').size().sort_values(ascending=False).plot(kind='barh')

## Duplicated Values

In [ ]:
df.duplicated().sum()

## Data Type info

In [ ]:
df.info()

### Data preparation 
- index column is redundant
- date column should be datetime index not string
- drop all the duplicates
- make day, month, year type of features

In [ ]:
df_copy = df.copy()
df_copy = df_copy.drop(columns=['index'])
df_copy = df_copy.dropna(axis=0)

In [ ]:
df_copy['date'] = pd.to_datetime(df_copy['date'],dayfirst=True)

df_copy['day'] = df_copy['date'].dt.day
df_copy['month'] = df_copy['date'].dt.month
df_copy['year'] = df_copy['date'].dt.year
df_copy['dayname'] = df_copy['date'].dt.day_name()
df_copy['dayofweek'] = df_copy['date'].dt.dayofweek
df_copy['quarter'] = df_copy['date'].dt.quarter

df_copy = df_copy.sort_values(by=['date','state_clean','district_clean','pincode'])
df_copy['pincode'] = df_copy['pincode'].apply(str)
df_copy.head()

# Insight Methodology - Spatio Temporal Analysis
1. Which states/districts/pincodes consistently generate the highest biometric volumne ?
2. How does activity vary by Day-of-week, Month/Quarter , School Calendar ?
3. Do activity spikes at month end ?
4. Under-served and Over-served districts
5. Age group behaviour, histograms , youth_ratio
6. Anomaly Detection - IQR, Rolling IQR or Isolation Forest
7. Forecasting(maybe)

## Feature generation

In [ ]:
df_copy['total_bio'] = df_copy['bio_age_5_17'] + df_copy['bio_age_17_']
df_copy['youth_ratio'] = df_copy['bio_age_5_17']/(df_copy['total_bio']+1)
df_copy['year_month'] = df_copy['date'].dt.to_period('M').astype(str)
df_copy['is_weekend'] = df_copy['dayofweek'].isin([5,6]).astype(int)

## Consistency Graphs

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,4))
consistency = df_copy.groupby('year_month').agg(
    total_districts = ('district_clean','nunique'),
    total_states= ('state_clean','nunique'),
    total_bio = ('total_bio','sum'),
    total_youth_bio  = ('bio_age_5_17','sum'),
    total_adult_bio = ('bio_age_17_','sum')
)

consistency[['total_districts','total_states']].plot(ax=ax[0])
ax[0].grid(alpha=0.3)

consistency[['total_bio','total_youth_bio','total_adult_bio']].plot(ax=ax[1])
ax[1].grid(alpha=0.3)

plt.tight_layout()

# Exploratory Data Analysis

## 1. Total National Biometric Volume per month

In [ ]:
fig,ax = plt.subplots(figsize=(12,4))
monthly = df_copy.groupby(['year_month'])['total_bio'].sum()
monthly_youth = df_copy.groupby(['year_month'])['bio_age_5_17'].sum()
monthly_adult = df_copy.groupby(['year_month'])['bio_age_17_'].sum()
monthly.plot(title='National Monthly Biometric Registration',marker='*',linestyle='--',color='black',ax=ax,label='Total')
monthly_youth.plot(marker='*',linestyle='--',color='blue',ax=ax,label='Youth')
monthly_adult.plot(marker='*',linestyle='--',color='red',ax=ax,label='Adult')
ax.grid(alpha=0.3)
ax.legend()
ax.set_yscale('log')
ax.tick_params('x',rotation=45)
ax.set_ylabel('log(Total Bio)')
plt.show()

### NOTE: This graph is misleading, because dips and peaks aren't due to naturally less volume in that time frame but due to availability of data at that time

- Peak in JULY is due to UIDAI changes in necessary biometric registration for children aged between 5 and 15

In [ ]:
fig,ax = plt.subplots(figsize=(12,4))

month_adjusted = (
    df_copy
    .groupby('year_month')
    .agg(
        total_bio = ('total_bio','sum'),
        days = ('date','nunique'),
        districts = ('district_clean','nunique')
    )
)

month_adjusted['bio_per_day_per_district'] = (
    month_adjusted['total_bio']/
    (month_adjusted['days']*month_adjusted['districts'])
)

month_adjusted['bio_per_day_per_district'].plot(ax=ax,color='black',marker='*',linestyle='--')
ax.grid()
ax.set_title('Total Bio Volume per day per district - normalized')

In [ ]:
month_adjusted

#### This graph can't be used for any interpretation, it just proves the point that initial months till July only have 1 day per state but the volume in that one day id comparable to the total volume of the months in later months - which means that those initial months were already aggregated over the month and sampled to the first date of that month

#### Event organizers gave no information regarding this, we hypothesis this and the validating evidence is acquired from the above shown data frame and graph, ex: March has about 8.3 million registrations for just one day when December had 8.6 million registrations for close to whole month

## 2. Weekday demand pattern (take with caution, only months with data for all the days of the month were used here)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,4),sharey=True)

day_wise_data = df_copy[df_copy.month >= 9]

weekday = day_wise_data.groupby('dayname')['total_bio'].mean().sort_values()
weekday_youth = day_wise_data.groupby('dayname')['bio_age_5_17'].mean().sort_values()
weekday_adult = day_wise_data.groupby('dayname')['bio_age_17_'].mean().sort_values()
weekday.plot(kind='barh',title='Avg Bio Vol (September till end)',ax=ax[0])
weekday_youth.plot(kind='barh',title='Avg Youth Bio Vol (September till end)',ax=ax[1])
weekday_adult.plot(kind='barh',title='Avg Adult Bio Vol (September till end)',ax=ax[2])

[ax[i].grid(alpha=0.1) for i in range(3)]
# [ax[i].set_xscale('log') for i in range(3)]

plt.tight_layout()

## Adding Weekdays trend for total volume across all the datasets

In [ ]:
enrolment_ = pd.read_csv('/kaggle/input/enrolment-cleaned-rig/Enrollment cleaned.csv')
# df_copy mera dataset
demo_ = pd.read_csv('/kaggle/input/demo-cleaned-sid/demo_cleaned.csv')

fig,ax = plt.subplots(1,3,figsize=(15,4))

weekday_bio = df_copy[df_copy.month >= 9].groupby('dayname')['total_bio'].mean().sort_values(ascending=False)
weekday_enrolment = enrolment_[enrolment_.month >= 9].groupby('dayname')['Total Registrations'].mean().sort_values(ascending=False)
weekday_demo = demo_[demo_.month >= 9].groupby('dayname')['total_demo'].mean().sort_values(ascending=False)

weekday_bio.plot(kind='barh',title='Avg Biometric Volume (September till end)',ax=ax[1])
weekday_enrolment.plot(kind='barh',title='Avg Enrolment Volume (September till end)',ax=ax[0])
weekday_demo.plot(kind='barh',title='Avg Demographic Volume (September till end)',ax=ax[2])

_ = [ax[i].invert_yaxis() for i in range(3)]
_ = [ax[i].set_ylabel('DayName') for i in range(3)]
plt.tight_layout()

## 3. Top regions by Biometric Volume

In [ ]:
k = 10
fig,ax = plt.subplots()
top_states = (
    df_copy.groupby('state_clean')['total_bio']
    .sum()
    .sort_values(ascending=False)
    .head(k)
)

top_states.plot(kind='barh',ax=ax)
# plt.gca().invert_yaxis()
ax.invert_yaxis()
ax.set_xscale('log')
ax.set_ylabel('State')
ax.set_title('Top {k} States by Total Biometric Volume'.format(k=k))

## Contribution of each state to National Total 

In [ ]:
top_states.reset_index()

In [ ]:
top_k=10
national_total = df_copy['total_bio'].sum()

top_contributing_states = (
    df_copy.groupby('state_clean').agg(
        pct = ('total_bio',lambda x: x.sum()/national_total)
    ).sort_values(by='pct',ascending=False).head(top_k)
)

top_contributing_states.plot(kind='barh')
plt.title('% Contribution by each state in the Total National Biometric Volume')
plt.xlabel('Percentage Contribution')
plt.ylabel('State')
plt.gca().invert_yaxis()

## 4. Youth Behaviour

In [ ]:
plt.figure(figsize=(12,4))
youth_trend = df_copy.groupby('year_month')['youth_ratio'].mean()
youth_trend.plot(kind='line',linestyle='--',color='black',marker='*',title='Youth Biometric Update Trend')
plt.ylabel('Youth Ratio(volume of youth bio / adult bio)')
plt.grid()

In [ ]:
yt_enrol = enrolment_.groupby('year_month')['youth_ratio'].mean()
yt_demo = demo_.groupby('year_month')['youth_ratio'].mean()

fig,ax = plt.subplots(figsize=(15,4))

youth_trend.plot(kind='line',linestyle='--',color='black',marker='*',label='biometric',ax=ax)
yt_enrol.plot(kind='line',linestyle='--',color='red',marker='*',label='enrolment',ax=ax)
yt_demo.plot(kind='line',linestyle='--',color='blue',marker='*',label='demographic',ax=ax)
ax.grid(alpha=0.3)
ax.set_title('Avg Youth Ratio')
ax.set_ylabel('Mean Youth Ratio(age between 5 and 17/total)')
ax.legend()

In [ ]:
enrolment_.head(3)

In [ ]:
enrol_monthly = (
    enrolment_
    .groupby('year_month')[['age_0_5','age_5_17','age_18_greater','Total Registrations']]
    .sum()
    .reset_index()
)

for col in ['age_0_5', 'age_5_17', 'age_18_greater']:
    enrol_monthly[f'{col}_share'] = enrol_monthly[col] / enrol_monthly['Total Registrations']

x = enrol_monthly['year_month']

y1 = enrol_monthly['age_0_5']
y2 = enrol_monthly['age_5_17']
y3 = enrol_monthly['age_18_greater']

plt.stackplot(
    x,
    y1,y2,y3,
    labels=['Child','Youth','Adult'],
    alpha=0.8
)

plt.legend(loc='upper left')
plt.title('Age composition of New Enrolments Over Time')
plt.ylabel('Share of Total Enrolments')
plt.xlabel('Month')

plt.xticks(rotation=45)
plt.tight_layout()

## 5. District with unstable demand (Adv Viz - 01)

In [ ]:
top_k=10

district_stats = (
    df_copy.groupby('district_clean')['total_bio']
    .agg(['std','mean'])
)

district_stats['volatility'] = district_stats['std'] / (district_stats['mean'] + 1)
# state_stats['volatility'] = state_stats['std'] / (state_stats['mean'] + 1)
min_max = MinMaxScaler()
district_stats = district_stats.sort_values(by=['volatility'],ascending=False).head(top_k)

volatility = min_max.fit_transform(district_stats['volatility'].values.reshape(-1,1))
label = district_stats.index.tolist()


fig,ax = plt.subplots(figsize=(15,6))
# district_stats['volatility'].sort_values(ascending=False).head(top_k).plot(kind='line',ax=ax,marker='*',linestyle='--',color='black',linewidth=1.5)
ax.plot(label,volatility,marker='*',linestyle='--',color='black')
ax.grid(alpha=0.3)
ax.tick_params('x',rotation=60)
ax.set_ylabel('Volatility')
ax.set_xlabel('Districts')
ax.set_title(f'{top_k} Most Volatile Districts for Biometric Volume')

## 6.1 STL Decomposition of the Time Series - Identifying the core components of a time series
- we wish to isolate anamolies on day basis using IQR on the residuals of the STL
- 

In [ ]:
ts = (
    df_copy
    .groupby('year_month')['total_bio']
    .sum()
)

# ts.index = ts.index.to_timestamp()
ts = ts.sort_index()

# from statsmodels.tsa.seasonal import STL

stl = STL(ts, period=12, robust=True)
res = stl.fit()

res.plot();


## 8. Risk Profiling

- identifying metrics that can segment areas to give relevant information like Infrastructure, Access, risk and under utilized zones when these metrics are treated as proxy indicators
- using KMeans, cluster them
- t-SNE to plot them

In [ ]:
g = df_copy.groupby(['state_clean','district_clean','year_month'])['total_bio'].sum().reset_index()

stats = (
    g.groupby(['state_clean','district_clean'])['total_bio']
    .agg(['mean','std','min','max'])
)

stats['volatility'] = stats['std']/(stats['mean']+1)

stats['range_ratio'] = (stats['max']-stats['min']) / (stats['mean']+1)

stats.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans,DBSCAN

X = stats[['mean','volatility','range_ratio']].fillna(0)

X_scaled = StandardScaler().fit_transform(X)

# stats['cluster_kmeans'] = KMeans(n_clusters=4,random_state=42).fit_predict(X_scaled)
# stats['cluster_dbscan'] = DBSCAN(n_jobs=-1).fit_predict(X_scaled)

# stats.head()


# # stats_scaled = pd.DataFrame(np.concatenate((X_scaled,stats['cluster'].values)),columns=[f'col_{idx+1}' for idx in range(X.shape[1]+1)])


In [ ]:
cluster_idx = []
inertia = []
for clusters in range(1,10):
    m = KMeans(n_clusters=clusters,random_state=42)
    m.fit(X_scaled)
    inertia.append(m.inertia_)
    cluster_idx.append(clusters)

plt.plot(cluster_idx,inertia)
plt.grid(alpha=0.3)

In [ ]:
stats['cluster_kmeans'] = KMeans(n_clusters=4,random_state=42).fit_predict(X_scaled)
stats['cluster_dbscan'] = DBSCAN(n_jobs=-1).fit_predict(X_scaled)
stats.head()

In [ ]:
import seaborn as sns

fig,ax = plt.subplots(2,3,figsize=(12,6))
fig.suptitle('Cluster specific variation in input features for the choice of clustering algorithm - Biometric Volume')
# for idx,cols in enumerate(['mean','volatility','range_ratio']):
#     sns.boxplot(
#         data=stats,
#         y = cols,
#         x = stats['cluster'],
#         ax=ax[idx]
#     )

get_cols = ['mean','volatility','range_ratio']

for j in range(3):
    sns.boxplot(
        data=stats,
        y = get_cols[j],
        x = stats['cluster_kmeans'],
        showmeans=True,
        ax = ax[0,j]
    )
    sns.boxplot(
        data=stats,
        y = get_cols[j],
        x = stats['cluster_dbscan'],
        showmeans=True,
        ax = ax[1,j]
    )


plt.tight_layout()

In [ ]:
## from the above graph, it's clear that the Kmeans cluster are more organized and convey information more clearly,
## we'll map them for labelling them in graph

CLUSTER_NAMES = {
    0:'Stable Mid-Demand',
    1:'High Capacity Region',
    2:'Volatile',
    3:'Under utilized stable'
}

CLUSTER_COLORS = {
    0:'tab:blue',
    1:'tab:green',
    2:'tab:red',
    3:'tab:purple'
}

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

INPUT = stats.fillna(0)

INPUT_scaled = StandardScaler().fit_transform(INPUT[['mean','volatility','range_ratio']])

tsne = TSNE().fit_transform(INPUT_scaled)
pca = PCA(n_components=2).fit_transform(INPUT_scaled)



# plt.subplot(2,2,3)
# plt.scatter(tsne[:,0],tsne[:,1],c=INPUT['cluster_dbscan'],cmap='viridis')
# plt.grid(alpha=.3)
# plt.title('t-SNE Cluster Representation - DBSCAN')

# plt.subplot(2,2,4)
# plt.scatter(pca[:,0],pca[:,1],c=INPUT['cluster_dbscan'],cmap='viridis')
# plt.grid(alpha=.3)
# plt.title('PCA Cluster Representation - DBSCAN')


plt.tight_layout()

- well, the one who said that t-sne with DBSCAN is a crime, was not lying !

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,4))

labels = INPUT['cluster_kmeans'].values

for c in sorted(set(labels)):
    mask = (labels==c)
    ax[0].scatter(
        tsne[mask,0],
        tsne[mask,1],
        s=40,
        alpha=0.8,
        c = CLUSTER_COLORS[c],
        label=CLUSTER_NAMES[c]
    )

    ax[1].scatter(
            pca[mask,0],
            pca[mask,1],
            s=40,
            alpha=0.8,
            c = CLUSTER_COLORS[c],
            label=CLUSTER_NAMES[c]
        )

ax[0].set_title('District Segmentation By Demand Pattern (KMeans and t-SNE)')
ax[0].set_xlabel('Dim-01')
ax[0].set_ylabel('Dim-02')
ax[0].legend(title='Cluster Type')
ax[0].grid(alpha=0.3)

ax[1].set_title('District Segmentation By Demand Pattern (KMeans and PCA)')
ax[1].set_xlabel('Component-01')
ax[1].set_ylabel('Component-02')
ax[1].legend(title='Cluster Type')
ax[1].grid(alpha=0.3)

plt.tight_layout()

In [ ]:
state_cluster_distribution = (
    INPUT.reset_index()
    .groupby(['state_clean','cluster_kmeans'])
    .size()
    .reset_index(name='count')
)

pivot = state_cluster_distribution.pivot(
    index='state_clean',
    columns='cluster_kmeans',
    values='count'
).fillna(0)


state_pct = pivot.div(pivot.sum(axis=1),axis=0)


state_pct.plot(kind='bar',stacked=True,figsize=(12,6),color=[CLUSTER_COLORS[c] for c in pivot.columns])

plt.legend([CLUSTER_NAMES[c] for c in pivot.columns], title="Cluster Type")
plt.title("Cluster Composition by State")
plt.ylabel("Percentage of Districts")
plt.tight_layout()
plt.show()

In [ ]:
RISK_CLUSTERS = [2]

# total districts per state
state_total = pivot.sum(axis=1)

# risky districts per state
state_risk = pivot[RISK_CLUSTERS].sum(axis=1)

# percentage
state_risk_pct = (state_risk / state_total).sort_values(ascending=False)


top_states = state_risk_pct.head(10)

plt.figure(figsize=(10,5))
top_states.plot(kind="bar", color="crimson")

plt.ylabel("% Districts in Risk Clusters")
plt.title("Top States by Share of Volatile Districts")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [ ]:
top_state_names = top_states.index

top_state_pct = state_pct.loc[top_state_names]

top_state_pct.plot(
    kind="bar",
    stacked=True,
    figsize=(12,6),
    color=[CLUSTER_COLORS[c] for c in top_state_pct.columns]
)

plt.legend([CLUSTER_NAMES[c] for c in top_state_pct.columns],
           title="Cluster Type",
           bbox_to_anchor=(1.02,1), loc="upper left")

plt.ylabel("Fraction of Districts")
plt.title("Cluster Composition of High-Risk States")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

- we identified clusters based on avg values of mean and standard deviation coupled with max min ratio per district for each state
- we plotted those clusters using dimensionality reduction algorithms to see how different clusters play out
- moreover we found the percentage of each cluster in all the states (normalized between 0 and 1)
- one can use this graph to see which states perform best in each identified cluster


## Cluster Naming Methodology (motivated from the box plot above)
1. Stable Mid-Demand (cluster_id=0): consistent mid level usage with a little bit of fluctuations but not so chaotic
2. High Capacity Region (cluster_id=1): mostly high mean but predictable, high capacity regions
3. Volatile (cluster_id=2): obvious by the peak in volatility
4. under utilized stat (cluster_id=3): low to mid demand based on the data, under utilized 

## 9. Total cluster distribution

In [ ]:
df_clustered = pd.merge(left=df_copy,right=INPUT,how='left',on = 'district_clean')[df_copy.columns.tolist()+['cluster_kmeans']]
df_clustered['cluster'] = df_clustered['cluster_kmeans'].map(CLUSTER_NAMES)


# fig,ax = plt.subplots(1,2,figsize=(12,3))
df_clustered.groupby(['cluster']).size().sort_values().plot(kind='bar')
plt.semilogy()
plt.show()

## 10. Cluster Distributions over the year

In [ ]:
year_cluster_distribution = (
    df_clustered
    .groupby(['year_month','cluster_kmeans'])
    .size()
    .reset_index(name='count')
)

pivot = year_cluster_distribution.pivot(
    index='year_month',
    columns='cluster_kmeans',
    values='count'
).fillna(0)


pivot_pct = pivot.div(pivot.sum(axis=1),axis=0)

pivot_pct.plot(kind='bar',stacked=True,figsize=(12,4),color=[CLUSTER_COLORS[c] for c in pivot.columns])

## 11. Priority districts to act on for volatility check 
- we define a custom risk_score based on mean, volatility and range_ratio

$PriorityScore = 0.4*Volatility + 0.4*RangeRatio + 0.2*(1-MeanVolume) $

In [ ]:
stats

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

stats[['vol_n','range_n','mean_n']] = scaler.fit_transform(
    stats[['volatility','range_ratio','mean']]
)

stats['priority_score'] = 0.4*stats['vol_n'] + 0.4*stats['range_n'] + 0.2*(1-stats['mean_n'])

stats.head()

In [ ]:
stats.reset_index()

In [ ]:
priority_districts_desc = (
    stats.reset_index().sort_values(by='priority_score',ascending=False)
    [['state_clean','district_clean','cluster_kmeans','priority_score']]
    .head(20)
)

priority_districts_asc = (
    stats.reset_index().sort_values(by='priority_score',ascending=True)
    [['state_clean','district_clean','cluster_kmeans','priority_score']]
    .head(20)
)

fig,ax = plt.subplots(1,2,figsize=(12,4))

ax[0].barh(y = priority_districts_desc.apply(lambda row: row['district_clean']+","+row['state_clean'],axis=1),width=priority_districts_desc['priority_score'])
ax[0].set_title('Top 20 districts with highest priority score')
ax[0].grid(alpha=0.3)
ax[0].invert_yaxis()

ax[1].barh(y = priority_districts_asc.apply(lambda row: row['district_clean']+","+row['state_clean'],axis=1),width=priority_districts_asc['priority_score'])
ax[1].set_title('Last 20 districts with lowest priority score')
ax[1].grid(alpha=0.3)
ax[1].invert_yaxis()

plt.tight_layout()

## 12. State Operational based on volatile cluster counts

In [ ]:
state_burden = (
    stats.groupby('state_clean')
    .agg(
        avg_risk = ('priority_score','mean'),
        volatile_share = ('cluster_kmeans',lambda x: (x==2).mean()),
        under_utilized_share = ('cluster_kmeans',lambda x: (x==3).mean())
    ).reset_index()
)

state_burden = state_burden.sort_values(by = 'avg_risk',ascending=False)

In [ ]:
y_labels = state_burden.state_clean.values[:15]
width = state_burden.avg_risk.values[:15]
volatile_pct = state_burden.volatile_share.values[:15]

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,6))

ax[0].barh(y=y_labels,width = width)
ax[0].invert_yaxis()
ax[0].set_title('Avg Risk Score by state')


ax[1].barh(y=y_labels,width = volatile_pct)
ax[1].invert_yaxis()
ax[1].set_title('Pct of Volatile Districts')
plt.tight_layout()